In [1]:
# import necessary libraries and modules
# Numpy is the only external library used

import numpy as np

from hamming import HammingCode, BitError
from models import BinaryMessage
from transmission import (
    str2bit,
    bit2str,
    pad_last_chunk,
    list_to_chunks,
    chunks_to_list,
    generate_single_bit_error,
    generate_multi_bit_error
)

# Hamming's Code

As part of this question, you are going to use linear error-correcting codes invented by Richard W. Hamming in 1950 to detect errors [1, p.211], [2]. Hamming invented these linear error-correcting codes to detect up to two-bit errors or one-bit errors without detection of uncorrected errors [3]. The linear error-correcting code that encodes four bits of data into seven bits by adding three parity bits. Hamming’s (7,4) algorithm [3] can either correct any single-bit error, or detect all single-bit and two-bit errors as further described in [3]. Error-correcting codes are widely adopted in many kinds of transmission (including WiFi, cell phones, communication with satellites and spacecraft, and digital television) and storage (RAM, disk drives, flash memory, CDs, etc.) [1, p.211].

Hamming discovered a code in which a four-bit message is transformed into a seven-bit codeword. The generator matrix (G), parity-check matrix (H) discovered by Hamming is shown in fig. 2 and the Hamming’s Decoder Matrix (R) as shown in fig. 3. An encoding of a 4-bit binary value (word) \( w \) is a 7-bit vector i.e. the codeword resulting from a matrix-vector product $ c_w = G * w $ [3].

## 1. Encode

 Write a simple Hamming encoder program in Python, which, when given a 4-bit binary value, returns the resulting 7-bit binary vector codeword. Also implement the parity check functionality to see if there are any errors, that is to check whether $ H * c_w = \vec{0} $ holds, where \( \vec{0} \) is zero vector.


First we create an instance of the HammingCode class.

This class holds all functionalities for encoding, decoding and detecting errors in a
message. The class takes two arguments: message_bits and mode. With the parameter
`message_bits` the user can specify the length of the message in bits. This parameter
defaults to 4, but can be changed to any positive integer. The mode parameter specifies
the mode of the Hamming code implementation. With `detect`, the hamming code only tries
to detect, but not correct any errors in the message. With `correct`, the hamming code
tries to also correct the errors if any are detected. This has some drawbacks, depicted
later.
The code-rate CR depicts the ratio of the message bits to the total number of bits. The
higer the code-rate the more efficient is the code. This also has some drawbacks, which
will be discussed later.

In [2]:
# Creating a hamming code object

h = HammingCode(message_bits=4, mode="detect")  # 4 bit message
h

(7,4)-HammingCode: CR=0.5714 (detect mode)

Now we create a message to encode. The message needs to be a binary message of length 4.

In [3]:
message = [1, 0, 1, 1]
message

[1, 0, 1, 1]

A TypeError is raised if the message does not fit the hamming code message length. In
this step the message is also converted to a `BinaryMessage` object. `BinaryMessage` is a
custom extension of the `numpy.ndarray` class. It is used to store binary messages. It 
checks the input for validity and raises a TypeError if the input is not binary.
The `encode()` method of the hamming code class encodes the message and returns the
codeword, by adding the parity bits to the message. A 4 bit message will be transformed into
a 7 bit message.

In [4]:
# Encoding a non-valid message: not binary

h.encode([1,2,3])

ValueError: Array must contain only 0s and 1s

In [5]:
# Encoding a non-valid message: not 4 bits

h.encode([1,0,1,1,1,1])

ValueError: Message must have 4 bits, but has 6 bits

To encode a message, the message vector is multiplied by the generator matrix (which is created when the hamming code class is instantiated).

In [6]:
# Encoding a valid message

codeword = h.encode(message)
codeword

BinaryMessage([1, 0, 1, 1, 0, 1, 0])

In [7]:
# The check_correct method checks if there are any errors in the codeword. In this case
# there are no errors.

codeword = h.check_correct(codeword)
codeword

BinaryMessage([1, 0, 1, 1, 0, 1, 0])

In [8]:
# The check_correct method checks if there are any errors in the codeword. In this case
# there is an error. The error is not corrected, but an `BitError` exception is raised.

codeword = h.simulate_1bit_flip(codeword)
codeword = h.check_correct(codeword)
codeword

BitError: 1- or 2-bit errors detected! Cannot correct in mode `detect`.

## 2. Decode

Create a decoder program in Python, which, when given a 7-bit vector codeword, returns the original 4-bit vector word. That is, if we are given a 4-bit word \( w \), and we apply our encoder to return a codeword $ c_w = G * w $, and then we apply the decoder matrix (R) (fig. 3) to $ c_w $, then it should return the original word, such that $ R * c_w = w $.

In [9]:
# Lets try to correct the error; switching to "correct" mode

h = HammingCode(message_bits=4, mode="correct")
codeword_corrected = h.check_correct(codeword)

In [11]:
# The corrected codeword is now decoded to retrieve the original message. The recovered
# message is in fact identical to the original message.

message_recovered = h.decode(codeword_corrected)
assert np.array_equal(message, message_recovered)
message_recovered

BinaryMessage([1, 0, 1, 1])

## 3. Error correction and detection

Test your code by creating a few 4-bit vectors and running encode and then decode to check if you end up with the original 4-bit vector. Also, test your code with some errors and see if the parity check can identify the errors if so, to what extent.

In the following cells the, the process of the hamming code implementation is shown. In the first cell, we simulate only one bit errors, assuming that two or more bit errors are not possible. For a number of tests a random message is generated, then decoded into a codeword using the hamming implementation and then a random bit is flipped. The hamming implementation will find that error and correct it as well as decode the codeword back into the original message. An assert statement checks if the decoded message equals the original message.

In [20]:
# 1-bit errors can be detected and corrected, hence we set the mode to "correct"
h = HammingCode(message_bits=4, mode="correct")
n_tests = 5

for i in range(n_tests):
    # generate random message
    message = np.random.randint(0, 2, 4)
    codeword = h.encode(message)
    print("Message:     ", message)
    print("Codeword:    ", codeword)

    # similuate transmission errors: only one bit is flipped
    codeword_transmitted = h.simulate_1bit_flip(codeword)
    print("Transmitted: ", codeword_transmitted)

    # check for errors and correct them if possible
    codeword_checked = h.check_correct(codeword_transmitted)
    message_recovered = h.decode(codeword_checked)
    print("Recovered:   ", message_recovered)
    print("_" * 36)
    assert np.array_equal(message, message_recovered)

Message:      [1 0 1 0]
Codeword:     [1 0 1 0 1 0 1]
Transmitted:  [0 0 1 0 1 0 1]
Recovered:    [1 0 1 0]
____________________________________
Message:      [1 1 1 1]
Codeword:     [1 1 1 1 1 1 1]
Transmitted:  [1 1 1 1 0 1 1]
Recovered:    [1 1 1 1]
____________________________________
Message:      [0 1 0 1]
Codeword:     [0 1 0 1 0 1 0]
Transmitted:  [1 1 0 1 0 1 0]
Recovered:    [0 1 0 1]
____________________________________
Message:      [0 0 1 0]
Codeword:     [0 0 1 0 1 1 0]
Transmitted:  [0 0 1 0 0 1 0]
Recovered:    [0 0 1 0]
____________________________________
Message:      [0 1 1 0]
Codeword:     [0 1 1 0 0 1 1]
Transmitted:  [1 1 1 0 0 1 1]
Recovered:    [0 1 1 0]
____________________________________


In the next cell, we do not assume anymore that only one bit errors are possible. On the likelihood of such errors, we need to decide whether to correct or only detect the errors. In this case The errors are only detected. The pipeline works as before: a random message is encoded, a bit flip is simulated and the implementation checks for errors. If there is an error it will raise an Exception. 

In [25]:
# 1- or 2-bit errors can be detected, but NOT corrected
h = HammingCode(message_bits=4, mode="detect")  # <- mode changed
n_tests = 5

for i in range(n_tests):
    # generate random message
    message = np.random.randint(0, 2, 4)
    codeword = h.encode(message)
    print("Message:     ", message)
    print("Codeword:    ", codeword)

    # similuate transmission errors: 0, 1 or 2 bits are flipped
    n_errors = np.random.randint(0, 3)
    if n_errors == 1:
        codeword_transmitted = h.simulate_1bit_flip(codeword)
    elif n_errors == 2:
        codeword_transmitted = h.simulate_2bit_flip(codeword)
    else:
        codeword_transmitted = codeword
    print("Transmitted: ", codeword_transmitted)

    # check for errors. If errors are detected, an exception is raised
    print(f"Number of errors: {n_errors}")
    try:
        codeword_checked = h.check_correct(codeword_transmitted)
        print("No errors detected.")
    except BitError as e:
        print(f"Error detected: {e}")
    print("_" * 36)

Message:      [1 1 0 1]
Codeword:     [1 1 0 1 0 0 1]
Transmitted:  [1 1 1 1 0 0 1]
Number of errors: 1
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
____________________________________
Message:      [0 1 1 0]
Codeword:     [0 1 1 0 0 1 1]
Transmitted:  [0 0 1 0 0 1 1]
Number of errors: 1
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
____________________________________
Message:      [1 1 1 1]
Codeword:     [1 1 1 1 1 1 1]
Transmitted:  [1 1 1 1 1 1 1]
Number of errors: 0
No errors detected.
____________________________________
Message:      [0 1 1 0]
Codeword:     [0 1 1 0 0 1 1]
Transmitted:  [1 1 0 0 0 1 1]
Number of errors: 2
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
____________________________________
Message:      [1 0 1 1]
Codeword:     [1 0 1 1 0 1 0]
Transmitted:  [1 0 1 1 0 1 0]
Number of errors: 0
No errors detected.
____________________________________


# Use-Case: Transmission over Noisy Channel

In the following we implemented a simple use-case, where a string message is decoded into a bit list representation and sent trough a noisy channel. Using hamming_protection the message reaches the recipient in a intact state, whereas without the hamming protection the sentence is destroyed.

### Assumption: only 1-bit errors possible

In [37]:
# Set a message to transmit.
message = "Hello World! I will transmit this message now. I hope it will arrive correctly."

# Activate hamming protection or not. If activated, the message will be encoded and
# corrected using hamming codes. If deactivated, the message will be transmitted as is
# and will likely contain errors.
hamming_protection = True

# Set the length of the packages. The message will be split into packages of this length.
# A usual length is 8. A shorter length protects the message to a higher degree, but also
# increases the overhead (reduced the code-rate or efficiency).
message_length = 8

# Set the probability of a bit flip. If hamming protection is activated, the probability
# is not relevant, since the hamming code can correct single bit flips.
# Note: we assume that there is no more than one bit flip per package.
error_probability = 0.1

In [38]:
print("Message:      " + message)

message_encoded = str2bit(message)
binary_list, padding_needed = pad_last_chunk(message_encoded, message_length) # create binary
message_binary = list_to_chunks(binary_list, message_length)

if hamming_protection:
    h = HammingCode(message_length, mode="correct")
    message_binary_encoded = []
    for row in message_binary:
        message_binary_encoded.append(h.encode(np.array(row)))
    message_binary = np.array(message_binary_encoded)

# simulate noise
error = generate_single_bit_error(BinaryMessage(message_binary), error_probability)
packages_transmitted = (message_binary + error) % 2
nb_errors = np.count_nonzero(error)
print("Nb Errors:   ", nb_errors)

if hamming_protection:
    message_binary_decoded = []
    for row in packages_transmitted:
        corrected = h.check_correct(np.array(row))
        message_binary_decoded.append(h.decode(corrected))
    packages_transmitted = np.array(message_binary_decoded)
message_flat = chunks_to_list(packages_transmitted)
message_flat = message_flat[:(None if padding_needed == 0 else -padding_needed)]

print("Transmitted: ", bit2str(message_flat))

assert message == bit2str(message_flat), "Message was not transmitted correctly!"

Message:      Hello World! I will transmit this message now. I hope it will arrive correctly.
Nb Errors:    6
Transmitted:  Hello World! I will transmit this message now. I hope it will arrive correctly.


### Multi bit errors

In the following cell, we do not assume that there are only one bit errors anymore. We set the hamming code mode to 'detect' to be able to detect 2 bit errors as well. Now we are not able to correct errors any more. In cases where errors are very rare and our package size is relatively small, we might risk try to correct errors and eventually not detect 2-bit errors.

In [39]:
# Set a message to transmit.
message = "Hello World! I will transmit this message now. I hope it will arrive correctly."

# Activate hamming protection or not.
hamming_protection = True

# Set the length of the packages
message_length = 8

# Set the probability of a bit flip
error_probability = 0.01

# Now we dont assume that there is only one bit flip per package. Depending on
# how probable errors are, we need to decide if we want to correct errors or only detect
# them. If we want to correct errors, we might get incorrect messages, because we cannot
# detect more than 1 bit errors. If we only want to detect errors, we can detect, but not
# correct, up to 2-bit errors per package. Set mode to "correct" or "detect" accordingly.
h = HammingCode(message_length, mode="detect")


# NOTE: in the above cenario, a good solution is reducing the package size. Using a
# package size of 4, the probability of 2-bit errors per package is much lower. Using the
# mode 'correct' in this case means that we will be able to transmit a correct message in
# most cases. It will not promise that the message will be transmitted correctly, but it
# is more likely. Just try it out :)

# message_length = 4
# h = HammingCode(message_length, mode="correct")

In [41]:
print("Message:      " + message)
message_encoded = str2bit(message)
binary_list, padding_needed = pad_last_chunk(message_encoded, message_length)
message_binary = list_to_chunks(binary_list, message_length)

if hamming_protection:
    message_binary_encoded = []
    for row in message_binary:
        message_binary_encoded.append(h.encode(np.array(row)))
    message_binary = np.array(message_binary_encoded)

# simulate noise
error = generate_multi_bit_error(BinaryMessage(message_binary), error_probability)
packages_transmitted = (message_binary + error) % 2
nb_errors = np.count_nonzero(error)
print("Nb Errors:   ", nb_errors)

if hamming_protection:
    message_binary_decoded = []
    for row in packages_transmitted:
        try:
            corrected = h.check_correct(np.array(row))
        except BitError as e:
            print(f"Error detected: {e}")
            corrected = row
        message_binary_decoded.append(h.decode(corrected))
    packages_transmitted = np.array(message_binary_decoded)
message_flat = chunks_to_list(packages_transmitted)
message_flat = message_flat[:(None if padding_needed == 0 else -padding_needed)]

print("Transmitted: ", bit2str(message_flat))

assert message == bit2str(message_flat), "Message was not transmitted correctly!"

Message:      Hello World! I will transmit this message now. I hope it will arrive correctly.
Nb Errors:    9
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Error detected: 1- or 2-bit errors detected! Cannot correct in mode `detect`.
Transmitted:  Hello World! I`sill transmit this message nou. I hope it will arrive correctly


AssertionError: Message was not transmitted correctly!